# ICOM
July 14, 2017

My attempt at directly talking to the RigExpert AA-54 through Python 3.

I'm really only interesting the FRX command, which reports the resistance and reactance of the antenna. The GET and GT commands from their documentation don't work.

To set this up, hook up the antenna to the AA-54, then hook up a serial cable from the computer to the AA-54, and finally turn on the AA-54. It should come up in PC mode. Then you can run this program to get the R and X data.

In [1]:
import serial

In [2]:
def read_string_response(io, maxchars=12000):
    """
    Read a response from the analyzer.
    
    Returns the string response if OK, or "ERROR" if
    there was an error.
    """
    response = ""
    for k in range(maxchars):
        c = io.read(1)
        #print(c)
        response += c.decode("UTF-8")
        #print(">%s<" % response)
        if response.endswith("ERROR"):
            return "Command error"
        if response.endswith("OK"):
            break
    return response

In [3]:
def convert_float(resp1):
    """
    Convert the string response to a list of floats
    """
    result = []
    resp = resp1[:-2]  # Remove "OK" at end
    lines = resp.splitlines()
    for instring in lines:
        if len(instring) > 0:
            x = []
            s = instring.replace("\n","").replace("\r","").split(",")
            if len(s) > 0:
                for sval in s:
                    x.append(float(sval))
                result.append(x)
    return result

In [4]:
def frx_command(io, n):
    cmd = b"FRX"+bytearray(str(n),"UTF-8")+b"\r"
    io.write(cmd)
    response = read_string_response(io)
    return [response, convert_float(response[:-2])]

In [5]:
def command(io, cmd):
    """
    Send a command to the analyzer and get the response.
    """
    io.write(cmd)
    response = read_string_response(io)
    return response.replace("\n","").replace("\r","")

In [6]:
portname = "/dev/ttyUSB0"
baud = 38400

In [7]:
io = serial.Serial(portname,
                   baud,
                   serial.EIGHTBITS,  # bytesize
                   serial.PARITY_NONE,
                   stopbits=serial.STOPBITS_ONE,
                   timeout=1000)

In [8]:
command(io, b"ON\r")

'OK'

In [9]:
command(io, b"FQ14076000\r")

'OK'

In [10]:
command(io, b"SW400000\r")

'OK'

In [11]:
result = frx_command(io, 100)

In [12]:
print(result[0])


13.876000,25.08,0.97
13.880000,25.23,1.22
13.883999,25.41,1.37
13.888000,26.01,1.64
13.892001,26.33,2.16
13.895999,26.67,2.42
13.899999,26.75,2.58
13.904000,27.40,2.76
13.908000,27.58,3.36
13.911999,28.12,3.81
13.916000,28.66,3.89
13.920001,28.63,4.18
13.923999,29.00,4.37
13.928001,29.52,4.51
13.931999,29.96,5.16
13.936000,30.22,5.17
13.939999,30.76,5.68
13.944000,30.95,5.84
13.948001,31.58,6.02
13.951999,32.18,6.50
13.956001,32.51,6.76
13.959999,33.32,6.76
13.963999,33.71,6.93
13.968000,33.89,7.53
13.972000,34.66,7.56
13.976000,34.87,7.81
13.979999,35.20,8.21
13.984001,36.30,8.50
13.987999,36.67,8.61
13.991999,37.13,8.95
13.996000,38.03,9.14
14.000000,38.34,9.34
14.004000,39.22,9.48
14.008001,39.69,9.71
14.012001,40.42,9.59
14.015999,41.29,10.17
14.020000,41.67,10.42
14.024000,42.51,10.08
14.027999,43.44,10.19
14.032000,43.70,10.39
14.036001,44.75,10.19
14.040000,45.58,10.66
14.043999,46.23,10.67
14.048000,47.37,10.52
14.052000,48.15,10.66
14.055999,48.82,10.65
14.060000,50.23,10.53


In [13]:
print(result[1])

[[13.876, 25.08, 0.97], [13.88, 25.23, 1.22], [13.883999, 25.41, 1.37], [13.888, 26.01, 1.64], [13.892001, 26.33, 2.16], [13.895999, 26.67, 2.42], [13.899999, 26.75, 2.58], [13.904, 27.4, 2.76], [13.908, 27.58, 3.36], [13.911999, 28.12, 3.81], [13.916, 28.66, 3.89], [13.920001, 28.63, 4.18], [13.923999, 29.0, 4.37], [13.928001, 29.52, 4.51], [13.931999, 29.96, 5.16], [13.936, 30.22, 5.17], [13.939999, 30.76, 5.68], [13.944, 30.95, 5.84], [13.948001, 31.58, 6.02], [13.951999, 32.18, 6.5], [13.956001, 32.51, 6.76], [13.959999, 33.32, 6.76], [13.963999, 33.71, 6.93], [13.968, 33.89, 7.53], [13.972, 34.66, 7.56], [13.976, 34.87, 7.81], [13.979999, 35.2, 8.21], [13.984001, 36.3, 8.5], [13.987999, 36.67, 8.61], [13.991999, 37.13, 8.95], [13.996, 38.03, 9.14], [14.0, 38.34, 9.34], [14.004, 39.22, 9.48], [14.008001, 39.69, 9.71], [14.012001, 40.42, 9.59], [14.015999, 41.29, 10.17], [14.02, 41.67, 10.42], [14.024, 42.51, 10.08], [14.027999, 43.44, 10.19], [14.032, 43.7, 10.39], [14.036001, 44.7

In [14]:
command(io, b"OFF\r")

'OK'

In [15]:
io.flush()
io.close()